# HNet-GPT: Structure-Aware Code Generation via Hierarchical Encoding and Transformer Decoding
Models available at: [your_github_link]/models/



# Section 0: Environment Setup & Dependencies
**First, install required packages:**

In [1]:
# Install required packages
!pip install datasets==2.16.1
!pip install huggingface_hub --upgrade
!pip install nltk
!pip install transformers>=4.21.0
!pip install radon

# Verify installations
import datasets
import transformers
print(f"Datasets version: {datasets.__version__}")
print(f"Transformers version: {transformers.__version__}")

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Foun

# Section 1: Introduction and Setup #

**Exploratory Research** - Novel hybrid architecture combining hierarchical encoding with GPT-2 generation

This notebook stats the discussion on the HNet-GPT architecture.

## Research Overview
This work introduces **HNet-GPT**, a novel hybrid architecture that combines:
- **Hierarchical Encoding (HNet)**: Structure-aware code understanding
- **Sequential Generation (GPT-2)**: Proven autoregressive text generation
- **Adaptive Fusion**: Smart integration of both approaches

## Key Results Preview
- **40.6%** better than Pure GPT-2 (8.20 vs 13.80 perplexity)
- **39.5%** better than Pure HNet (8.20 vs 13.56 perplexity)

## Architecture Comparison
Here are evaluated three fundamental approaches:
1. **Pure HNet**: End-to-end hierarchical transformer
2. **HNet-GPT2 Hybrid**: The novel approach
3. **Pure GPT-2**: Sequential transformer baseline


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs("/content/drive/MyDrive/hnet_gpt2_models", exist_ok=True)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Model, GPT2Config, AutoTokenizer
import time
import math
from tqdm.auto import tqdm
import json
import ast
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')
import re
import ast
import sys
from io import StringIO
import contextlib
import re
import numpy as np

import radon.complexity as radon_complexity
from radon.visitors import ComplexityVisitor

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


 # Section 2: Core Utilities & Helper Functions

This section contains essential utility functions used across all models:
- **Top-k/Top-p filtering**: Advanced text generation sampling
- **Data loading utilities**: MBPP dataset integration
* **Code-focused evaluation**: Code token identification, pattern recognition, and syntax validation


In [3]:
# Top-k/Top-p filtering function
def top_k_top_p_filtering(logits, top_k=0, top_p=1.0, filter_value=-float('Inf'), min_tokens_to_keep=1):
    """
    Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
    Args:
        logits: logits distribution shape (batch_size, vocabulary size)
        top_k: (int or float) If set to int > 0, only the top_k least likely tokens are kept per batch item.
               If set to float (0.0 < top_k < 1.0), the number of tokens kept per batch item represents
               the percentage of the vocabulary size (e.g. 0.2 means keep 20% of the vocabulary).
        top_p: (float) If set to < 1.0, only the most likely tokens with probabilities that add up to >= top_p are
               kept for generation.
        filter_value: (float) value that will be used to fill filtered tokens.
        min_tokens_to_keep: (int) Minimum number of tokens that cannot be filtered.
    """
    if top_k > 0:
        if not isinstance(top_k, int):
            top_k = int(top_k * logits.shape[-1]) # Convert to absolute number of tokens
        top_k = min(max(top_k, min_tokens_to_keep), logits.size(-1)) # Clamp to (min_tokens_to_keep, vocab_size)
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p < 1.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold (token with 0 are kept)
        sorted_indices_to_remove = cumulative_probs > top_p
        if min_tokens_to_keep > 1:
            # Keep at least min_tokens_to_keep (set to false the first min_tokens_to_keep)
            sorted_indices_to_remove[..., :min_tokens_to_keep] = 0
        # Shift the indices to the right to keep the first token above the threshold
        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits



# Evaluation functions
def check_syntax_validity(code_text):
    """Check if generated code is syntactically valid Python"""
    try:
        ast.parse(code_text)
        return True
    except SyntaxError:
        return False


# 1. CODE TOKEN IDENTIFICATION FUNCTION
# Add this after your existing utility functions in Section 2

def get_comprehensive_code_tokens(tokenizer):
    """Identify all code-relevant tokens in the vocabulary"""

    print("Identifying code-relevant tokens...")

    # All possible code elements
    code_elements = [
        # Single characters
        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',

        # Operators and symbols
        '+', '-', '*', '/', '%', '=', '==', '!=', '>', '<', '>=', '<=',
        '(', ')', '[', ']', '{', '}', '.', ',', ':', ';', '_', '"', "'",

        # Whitespace
        ' ', '\n', '\t',

        # Keywords
        'return', 'if', 'else', 'elif', 'for', 'while', 'def', 'class',
        'True', 'False', 'None', 'and', 'or', 'not', 'in', 'is',

        # Built-in functions
        'len', 'str', 'int', 'float', 'bool', 'list', 'dict', 'set',
        'range', 'sum', 'max', 'min', 'abs', 'round',
        'print', 'input', 'open',

        # Common method names
        'upper', 'lower', 'strip', 'split', 'join', 'replace', 'append', 'remove',

        # With spaces (GPT-2 tokenizer includes spaces)
        ' +', ' -', ' *', ' /', ' %', ' =', ' ==', ' !=', ' >', ' <', ' >=', ' <=',
        ' a', ' b', ' c', ' x', ' y', ' z', ' n', ' i', ' j', ' k',
        ' return', ' if', ' else', ' True', ' False', ' None',
        ' 0', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9',
    ]

    # Get token IDs
    code_token_ids = set()
    for element in code_elements:
        try:
            token_ids = tokenizer.encode(element, add_special_tokens=False)
            code_token_ids.update(token_ids)
        except:
            continue

    # Add common combinations that work
    common_combinations = [
        'a +', 'b +', 'x *', 'n %', '== 0', '> 0', '< 0',
        'return a', 'return x', 'return n', 'return True', 'return False',
        'len(', 'str(', 'int(', 'abs(', 'x)', 'a)', 'b)', 'n)',
    ]

    for combo in common_combinations:
        try:
            token_ids = tokenizer.encode(combo, add_special_tokens=False)
            code_token_ids.update(token_ids)
        except:
            continue

    code_token_list = sorted(list(code_token_ids))
    print(f"Identified {len(code_token_list)} code-relevant tokens")

    # Test coverage for common code
    test_code = "a + b"
    test_tokens = tokenizer.encode(test_code, add_special_tokens=False)
    overlap = len(set(test_tokens) & set(code_token_list))
    print(f"   Test coverage 'a + b': {overlap}/{len(test_tokens)} tokens ({overlap/len(test_tokens):.1%})")

    return code_token_list

# ============================================================================
# 2. CODE-FOCUSED EVALUATION FUNCTION
# Add this function to evaluate your existing models
# ============================================================================

def evaluate_with_code_focus(model, tokenizer, device, code_token_ids, model_name="Model"):
    """Evaluate existing models with code-focused approach"""

    print(f"\nCODE-FOCUSED EVALUATION: {model_name}")
    print("=" * 50)

    # Test prompts for code completion
    test_cases = [
        {
            'prompt': 'def add(a, b):\n    return ',
            'expected_patterns': ['a', 'b', '+', 'a + b'],
            'description': 'Basic addition'
        },
        {
            'prompt': 'def is_even(n):\n    return ',
            'expected_patterns': ['n', '%', '2', '==', '0', 'n % 2 == 0'],
            'description': 'Boolean logic'
        },
        {
            'prompt': 'def max_two(x, y):\n    if x > y:\n        return ',
            'expected_patterns': ['x', 'y', 'if', 'else'],
            'description': 'Conditional logic'
        },
        {
            'prompt': 'def square(n):\n    return ',
            'expected_patterns': ['n', '*', 'n * n'],
            'description': 'Simple expression'
        },
        {
            'prompt': 'def is_positive(x):\n    return ',
            'expected_patterns': ['x', '>', '0', 'x > 0'],
            'description': 'Comparison'
        }
    ]

    model.eval()

    # Results tracking
    total_tests = len(test_cases)
    syntax_successes = 0
    pattern_successes = 0
    code_token_successes = 0

    code_token_ids_set = set(code_token_ids)

    with torch.no_grad():
        for i, test_case in enumerate(test_cases, 1):
            print(f"\nTest {i}: {test_case['description']}")
            print(f"   Prompt: {test_case['prompt'].strip()}")

            prompt_tokens = tokenizer(test_case['prompt'], return_tensors='pt').to(device)

            # Try different generation strategies
            best_completion = ""
            best_score = 0

            strategies = [
                {"temperature": 0.1, "do_sample": False, "name": "Greedy"},
                {"temperature": 0.3, "do_sample": True, "top_k": 20, "name": "Conservative"},
                {"temperature": 0.7, "do_sample": True, "top_k": 50, "name": "Creative"}
            ]

            for strategy in strategies:
                try:
                    # Generate with existing model method
                    if hasattr(model, 'generate'):
                        generated = model.generate(
                            prompt_tokens.input_ids,
                            max_length=prompt_tokens.input_ids.size(1) + 15,
                            repetition_penalty=1.1,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.eos_token_id,
                            **{k: v for k, v in strategy.items() if k != 'name'}
                        )
                    else:
                        # Fallback for models without generate method
                        continue

                    full_text = tokenizer.decode(generated[0], skip_special_tokens=True)
                    completion = full_text[len(test_case['prompt']):].strip()

                    if completion and len(completion) > len(best_completion):
                        best_completion = completion

                except Exception as e:
                    continue

            print(f"   Generated: '{best_completion}'")

            if not best_completion:
                print(f"No generation produced")
                continue

            # Test 1: Syntax validity
            try:
                full_code = test_case['prompt'] + best_completion
                ast.parse(full_code)
                print(f"Valid syntax")
                syntax_successes += 1
            except SyntaxError:
                print(f"Syntax error")

            # Test 2: Contains expected patterns
            pattern_found = False
            for pattern in test_case['expected_patterns']:
                if pattern in best_completion:
                    print(f"Contains expected pattern: '{pattern}'")
                    pattern_found = True
                    break

            if pattern_found:
                pattern_successes += 1
            else:
                print(f"No expected patterns found")

            # Test 3: Uses code tokens
            completion_tokens = tokenizer.encode(best_completion, add_special_tokens=False)
            code_token_overlap = len(set(completion_tokens) & code_token_ids_set)
            code_token_ratio = code_token_overlap / len(completion_tokens) if completion_tokens else 0

            print(f"Code token usage: {code_token_overlap}/{len(completion_tokens)} ({code_token_ratio:.1%})")

            if code_token_ratio >= 0.5:  # At least 50% code tokens
                print(f"Good code token usage")
                code_token_successes += 1
            else:
                print(f"Low code token usage")

    # Calculate final scores
    syntax_rate = syntax_successes / total_tests
    pattern_rate = pattern_successes / total_tests
    code_token_rate = code_token_successes / total_tests

    # Composite score (weighted)
    composite_score = (syntax_rate * 0.4 + pattern_rate * 0.4 + code_token_rate * 0.2)

    print(f"\nCODE-FOCUSED RESULTS FOR {model_name}:")
    print(f"Syntax Validity: {syntax_successes}/{total_tests} ({syntax_rate:.1%})")
    print(f"Pattern Recognition: {pattern_successes}/{total_tests} ({pattern_rate:.1%})")
    print(f"Code Token Usage: {code_token_successes}/{total_tests} ({code_token_rate:.1%})")
    print(f"Composite Score: {composite_score:.1%}")

    return {
        'syntax_rate': syntax_rate,
        'pattern_rate': pattern_rate,
        'code_token_rate': code_token_rate,
        'composite_score': composite_score
    }

# ============================================================================
# 3. ENHANCED RESULTS COMPARISON
# Add this to compare all your existing models
# ============================================================================

def show_enhanced_results():
    """Show enhanced evaluation of your existing trained models"""

    print("ENHANCED EVALUATION OF EXISTING MODELS")
    print("=" * 60)
    print("Testing your trained models with code-focused evaluation")

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    vocab_size = tokenizer.vocab_size

    # Get code tokens
    code_token_ids = get_comprehensive_code_tokens(tokenizer)

    # Test your existing models
    models_to_test = [
        {
            'name': 'Pure HNet',
            'creator': lambda: create_pure_hnet_model(vocab_size, tokenizer),
            'file': 'Pure_HNet.pt'
        },
        {
            'name': 'HNet-GPT2-Hybrid',
            'creator': lambda: create_hnet_gpt2_hybrid(vocab_size, tokenizer),
            'file': 'HNet-GPT2-Hybrid.pt'
        },
        {
            'name': 'Pure GPT-2',
            'creator': lambda: create_pure_gpt2_baseline(vocab_size),
            'file': 'Pure_GPT-2.pt'
        }
    ]

    results = []

    for model_config in models_to_test:
        print(f"\n{'='*60}")
        print(f"TESTING: {model_config['name']}")
        print(f"{'='*60}")

        try:
            # Create model
            model = model_config['creator']().to(device)

            # Load existing weights
            model_path = f"/content/drive/MyDrive/hnet_gpt2_models/{model_config['file']}"
            model.load_state_dict(torch.load(model_path, map_location=device))
            print(f"Loaded existing model weights")

            # Enhanced evaluation
            enhanced_results = evaluate_with_code_focus(
                model, tokenizer, device, code_token_ids, model_config['name']
            )

            # Store results
            result = {
                'name': model_config['name'],
                'loaded': True,
                **enhanced_results
            }
            results.append(result)

            # Clear GPU memory
            del model
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"Failed to test {model_config['name']}: {e}")
            results.append({
                'name': model_config['name'],
                'loaded': False,
                'composite_score': 0.0
            })

    # Show final comparison
    print(f"\nENHANCED RESULTS COMPARISON")
    print("=" * 50)
    print(f"{'Model':<20} {'Syntax':<8} {'Patterns':<10} {'Tokens':<8} {'Overall':<8}")
    print("-" * 55)

    # Sort by composite score
    sorted_results = sorted([r for r in results if r['loaded']],
                           key=lambda x: x['composite_score'], reverse=True)

    for i, result in enumerate(sorted_results, 1):
        emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
        print(f"{emoji} {result['name']:<18} {result['syntax_rate']:<8.1%} "
              f"{result['pattern_rate']:<10.1%} {result['code_token_rate']:<8.1%} "
              f"{result['composite_score']:<8.1%}")

    if sorted_results:
        winner = sorted_results[0]
        print(f"\nBEST PERFORMING MODEL: {winner['name']}")
        print(f"   Overall Score: {winner['composite_score']:.1%}")

        if winner['composite_score'] > 0.5:
            print(f"STRONG PERFORMANCE! Model shows good code generation capability")
        elif winner['composite_score'] > 0.25:
            print(f"MODERATE PERFORMANCE! Model shows some code awareness")
        else:
            print(f"LEARNING OPPORTUNITY! Model needs code-focused training")

    return results

 # Section 3: Data Loading & Preprocessing #
 This section handles the MBPP (Mostly Basic Python Programs) dataset:
- **Primary**: CodeSearchNet Python dataset (4,000 train + 800 test examples)
- **Fallback**: High-quality synthetic Python code patterns
- **Format**: Task description + code solution pairs
- **Preprocessing**: Tokenization with GPT-2 tokenizer

In [4]:
# Try to import datasets, but don't fail if it's not available
try:
    from datasets import load_dataset
    DATASETS_AVAILABLE = True
except ImportError:
    DATASETS_AVAILABLE = False
    print("Datasets library not available, will use synthetic data")



def create_mbpp_data_loaders(tokenizer, device, max_length=512, batch_size=4):
    """Create data loaders for MBPP dataset or synthetic code data"""

    class CodeDataset(Dataset):
        def __init__(self, split='train', max_length=512):
            self.examples = []
            data_loaded = False

            # Approach 1: Try loading CodeSearchNet Python (now with correct field names!)
            if DATASETS_AVAILABLE and not data_loaded:
                try:
                    print(f"Attempting to load CodeSearchNet Python {split} split...")
                    dataset = load_dataset('code_search_net', 'python', split=split, trust_remote_code=True)

                    count = 0
                    max_examples = 4000 if split == 'train' else 800  # Double the data

                    for item in tqdm(dataset, desc=f"Processing CodeSearchNet {split} data"):
                        if count >= max_examples:
                            break

                        # Use the CORRECT field names from the debug output
                        code = item.get('func_code_string', '').strip()
                        func_name = item.get('func_name', '').strip()
                        docstring = item.get('func_documentation_string', '').strip()

                        # Basic filtering for valid Python functions
                        if code and len(code) > 30 and 'def ' in code:
                            # Create a task-like format
                            if docstring and len(docstring) > 5:
                                text = f"# Task: {docstring}\n\n# Solution:\n{code}"
                            else:
                                text = f"# Function: {func_name}\n\n# Solution:\n{code}"

                            # Tokenize
                            tokens = tokenizer(
                                text,
                                truncation=True,
                                max_length=max_length,
                                padding='max_length',
                                return_tensors='pt'
                            )

                            self.examples.append({
                                'input_ids': tokens['input_ids'].squeeze(0),
                                'labels': tokens['input_ids'].squeeze(0).clone()
                            })
                            count += 1

                    if len(self.examples) > 0:
                        data_loaded = True
                        print(f"Successfully loaded {len(self.examples)} CodeSearchNet examples for {split}")
                    else:
                        print(f"No valid examples found in CodeSearchNet {split}")

                except Exception as e:
                    print(f"Failed to load CodeSearchNet: {e}")

            # Approach 2: Create high-quality synthetic Python code data


            ###
            if not data_loaded:
                print(f"Creating synthetic Python code data for {split}...")
                num_examples = 500 if split == 'train' else 100

                # More diverse and realistic code patterns
                code_patterns = [
                    # Function definitions
                    "def calculate_sum(numbers):\n    '''Calculate sum of a list'''\n    total = 0\n    for num in numbers:\n        total += num\n    return total",

                    # Classes
                    "class DataProcessor:\n    def __init__(self, data):\n        self.data = data\n        self.processed = False\n    \n    def process(self):\n        self.data = [x * 2 for x in self.data]\n        self.processed = True",

                    # List comprehensions and algorithms
                    "def find_primes(n):\n    '''Find all prime numbers up to n'''\n    primes = []\n    for num in range(2, n + 1):\n        is_prime = True\n        for i in range(2, int(num ** 0.5) + 1):\n            if num % i == 0:\n                is_prime = False\n                break\n        if is_prime:\n            primes.append(num)\n    return primes",

                    # Recursion
                    "def fibonacci(n):\n    '''Calculate nth Fibonacci number'''\n    if n <= 1:\n        return n\n    return fibonacci(n - 1) + fibonacci(n - 2)",

                    # String manipulation
                    "def reverse_words(sentence):\n    '''Reverse words in a sentence'''\n    words = sentence.split()\n    reversed_words = words[::-1]\n    return ' '.join(reversed_words)",

                    # Dictionary operations
                    "def count_frequencies(items):\n    '''Count frequency of each item'''\n    freq_dict = {}\n    for item in items:\n        if item in freq_dict:\n            freq_dict[item] += 1\n        else:\n            freq_dict[item] = 1\n    return freq_dict",

                    # Error handling
                    "def safe_divide(a, b):\n    '''Safely divide two numbers'''\n    try:\n        result = a / b\n        return result\n    except ZeroDivisionError:\n        return None\n    except TypeError:\n        return 'Invalid input types'",

                    # Sorting algorithms
                    "def bubble_sort(arr):\n    '''Implement bubble sort'''\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n - i - 1):\n            if arr[j] > arr[j + 1]:\n                arr[j], arr[j + 1] = arr[j + 1], arr[j]\n    return arr",
                ]

                for i in range(num_examples):
                    # Rotate through patterns and add variations
                    base_code = code_patterns[i % len(code_patterns)]

                    # Add task description
                    task_descriptions = [
                        "Write a function to solve this problem",
                        "Implement a solution for the following",
                        "Create a Python function that handles this task",
                        "Develop an algorithm to compute the result",
                    ]

                    task = task_descriptions[i % len(task_descriptions)]
                    full_text = f"# Task: {task}\n\n{base_code}"

                    # Tokenize
                    tokens = tokenizer(
                        full_text,
                        truncation=True,
                        max_length=max_length,
                        padding='max_length',
                        return_tensors='pt'
                    )

                    self.examples.append({
                        'input_ids': tokens['input_ids'].squeeze(0),
                        'labels': tokens['input_ids'].squeeze(0).clone()
                    })

                print(f"Created {len(self.examples)} synthetic examples for {split}")

        def __len__(self):
            return len(self.examples)

        def __getitem__(self, idx):
            return self.examples[idx]['input_ids'], self.examples[idx]['labels']

    # Create datasets
    print("\nPreparing datasets...")
    train_dataset = CodeDataset('train', max_length)
    test_dataset = CodeDataset('test', max_length)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  # Avoid multiprocessing issues
        pin_memory=True if device == 'cuda' else False
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True if device == 'cuda' else False
    )

    print(f"\nDataset Statistics:")
    print(f"   Training examples: {len(train_dataset)}")
    print(f"   Test examples: {len(test_dataset)}")
    print(f"   Batch size: {batch_size}")
    print(f"   Max sequence length: {max_length}")

    return train_loader, test_loader

# Section 4: Model Architectures

##Pure GPT-2 Baseline Model

Standard sequential transformer architecture:
- **12-layer GPT-2** configuration
- **Sequential processing** of code tokens
- **No hierarchical structure**
- **Baseline performance** for comparison

In [5]:
def create_pure_gpt2_baseline(vocab_size):
    """Pure GPT-2 baseline for comparison"""

    class PureGPT2(nn.Module):
        def __init__(self, vocab_size, embed_dim=768):
            super().__init__()

            # Create GPT-2 configuration
            self.config = GPT2Config(
                vocab_size=vocab_size,
                n_embd=embed_dim,
                n_layer=12,
                n_head=12,
                activation_function='gelu_new',
                resid_pdrop=0.1,
                embd_pdrop=0.1,
                attn_pdrop=0.1,
            )

            # Create GPT-2 model
            self.gpt2 = GPT2Model(self.config)

            # Output projection
            self.output = nn.Linear(embed_dim, vocab_size)

        def forward(self, input_ids, labels=None):
            # Forward through GPT-2
            outputs = self.gpt2(input_ids)
            hidden_states = outputs.last_hidden_state

            # Project to vocabulary
            logits = self.output(hidden_states)

            # Compute loss
            loss = None
            if labels is not None:
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = labels[..., 1:].contiguous()
                loss_fn = nn.CrossEntropyLoss()
                loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            return logits, loss

            ##
        def generate(self, input_ids, max_length=None, num_return_sequences=1, temperature=1.0, do_sample=False, **kwargs):
            """FIXED: Use parameter instead of tokenizer reference"""
            self.eval()
            device = input_ids.device

            if max_length is None:
                max_length = input_ids.size(1) + 50

            max_length = min(max_length, 512)
            generated = input_ids.clone()

            top_k = kwargs.get('top_k', 50)
            top_p = kwargs.get('top_p', 1.0)
            repetition_penalty = kwargs.get('repetition_penalty', 1.2)
            eos_token_id = kwargs.get('eos_token_id', 50256)  # FIXED: Get EOS token from kwargs

            with torch.no_grad():
                for _ in range(max_length - input_ids.size(1)):
                    logits, _ = self.forward(generated)
                    next_token_logits = logits[:, -1, :]

                    # Apply repetition penalty
                    if repetition_penalty != 1.0 and generated.size(1) > 0:
                        for i in range(generated.size(0)):
                            for prev_token_idx in set(generated[i, max(0, generated.size(1) - 10):].tolist()):
                                if prev_token_idx < next_token_logits.size(-1):
                                    if next_token_logits[i, prev_token_idx] < 0:
                                        next_token_logits[i, prev_token_idx] *= repetition_penalty
                                    else:
                                        next_token_logits[i, prev_token_idx] /= repetition_penalty

                    next_token_logits = next_token_logits / max(temperature, 1e-8)

                    if do_sample:
                        filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)

                        all_filtered_out = (filtered_logits == -float('Inf')).all(dim=-1)
                        if all_filtered_out.any():
                            for i in range(all_filtered_out.size(0)):
                                if all_filtered_out[i]:
                                    filtered_logits[i, 0] = 1.0

                        filtered_logits = torch.nan_to_num(filtered_logits, nan=-1e9, posinf=1e9, neginf=-1e9)
                        probs = F.softmax(filtered_logits, dim=-1)
                        probs = torch.clamp(probs, min=1e-9)
                        probs = probs / probs.sum(dim=-1, keepdim=True)
                        next_token = torch.multinomial(probs, num_samples=1)
                    else:
                        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)

                    generated = torch.cat([generated, next_token], dim=-1)

                    # FIXED: Use eos_token_id parameter instead of tokenizer
                    if eos_token_id is not None and (next_token == eos_token_id).any():
                        break

                    if generated.size(1) >= input_ids.size(1) + 3:
                        if (generated[:, -1] == generated[:, -2]).any() and \
                          (generated[:, -1] == generated[:, -3]).any():
                            break

            return generated

            ##



    return PureGPT2(vocab_size)

##Pure HNet: End-to-End Hierarchical Architecture

Novel hierarchical transformer with 4-level processing:
1. **Chunk-Level Processing**: Fixed overlapping windows with attention pooling
2. **Global Context**: Inter-chunk relationship modeling  
3. **Hierarchical-Sequential Bridge**: Project back to sequence space
4. **Final Sequence Processing**: Custom transformer layers

**Key Innovations:**
- Attention-based chunk pooling (vs mean pooling)
- Explicit global context stage
- End-to-end hierarchical optimization

In [6]:
def create_pure_hnet_model(vocab_size, tokenizer):
    """
    Create Pure HNet model - End-to-end hierarchical transformer

    Architecture Philosophy:
    - Multi-level hierarchical processing (chunk → global → sequence)
    - Attention-based chunk pooling for better representations
    - Gated fusion between hierarchical and sequential features
    - Designed specifically for structured text like code
    """

    class PureHNetModel(nn.Module):
        def __init__(self, vocab_size, tokenizer, embed_dim=768):
            super().__init__()
            self.tokenizer = tokenizer
            self.embed_dim = embed_dim

            print(f"Building Pure HNet Architecture:")
            print(f"Vocabulary: {vocab_size:,} tokens")
            print(f"Embedding dimension: {embed_dim}")

            # ============= Core Embeddings =============
            self.embed = nn.Embedding(vocab_size, embed_dim)
            self.pos_embed = nn.Embedding(1024, embed_dim)
            self.dropout = nn.Dropout(0.1)

            # ============= Hierarchical Parameters =============
            self.num_chunks = 32  # Number of hierarchical chunks
            self.chunk_size = 512 // self.num_chunks
            self.chunk_overlap = int(self.chunk_size * 0.25)  # 25% overlap

            print(f"Chunking strategy: {self.num_chunks} chunks of size {self.chunk_size}")
            print(f"Chunk overlap: {self.chunk_overlap} tokens")

            # ============= Level 1: Chunk-Level Processing =============
            chunk_encoder_layer = nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=12,
                dim_feedforward=embed_dim * 4,
                dropout=0.1,
                activation='gelu',
                batch_first=True
            )
            self.chunk_encoder = nn.TransformerEncoder(chunk_encoder_layer, num_layers=6)

            # ============= Level 2: Global Context Processing =============
            global_encoder_layer = nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=12,
                dim_feedforward=embed_dim * 4,
                dropout=0.1,
                activation='gelu',
                batch_first=True
            )
            self.global_encoder = nn.TransformerEncoder(global_encoder_layer, num_layers=4)

            # ============= Level 3: Hierarchical-to-Sequential Bridge =============
            self.chunk_projection = nn.Sequential(
                nn.Linear(embed_dim, embed_dim),
                nn.GELU(),
                nn.Dropout(0.1),
                nn.Linear(embed_dim, embed_dim)
            )

            # Gating mechanism for hierarchical fusion
            self.hierarchical_gate = nn.Sequential(
                nn.Linear(embed_dim * 2, embed_dim),
                nn.Dropout(0.1),
                nn.Sigmoid()
            )

            # ============= Level 4: Final Sequence Processing =============
            sequence_encoder_layer = nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=12,
                dim_feedforward=embed_dim * 4,
                dropout=0.1,
                activation='gelu',
                batch_first=True
            )
            self.sequence_encoder = nn.TransformerEncoder(sequence_encoder_layer, num_layers=6)

            # ============= Output Layer =============
            self.output = nn.Linear(embed_dim, vocab_size)

            # Initialize weights
            self.apply(self._init_weights)

            total_params = sum(p.numel() for p in self.parameters())
            print(f"Total parameters: {total_params:,}")

        def _init_weights(self, module):
            """Initialize model weights with small random values"""
            if isinstance(module, (nn.Linear, nn.Embedding)):
                module.weight.data.normal_(mean=0.0, std=0.02)
                if isinstance(module, nn.Linear) and module.bias is not None:
                    module.bias.data.zero_()

        def _create_hierarchical_chunks(self, hidden_states, input_ids):
            """
            Create hierarchical chunks with smart overlapping

            Key Innovation: Attention-based pooling instead of mean pooling
            for better chunk representations
            """
            B, L, D = hidden_states.shape
            chunks = []

            for batch_idx in range(B):
                batch_chunks = []

                # Create overlapping chunks for better context
                for i in range(self.num_chunks):
                    start = i * self.chunk_size
                    end = min(start + self.chunk_size + self.chunk_overlap, L)

                    if start < L:
                        chunk_tokens = hidden_states[batch_idx, start:end]
                        # Use attention pooling for better representations
                        chunk_repr = self._attention_pool(chunk_tokens)
                        batch_chunks.append(chunk_repr)

                # Ensure consistent chunk count
                while len(batch_chunks) < self.num_chunks:
                    batch_chunks.append(torch.zeros(D, device=hidden_states.device))

                batch_chunks = batch_chunks[:self.num_chunks]
                chunks.append(torch.stack(batch_chunks))

            return torch.stack(chunks, dim=0)

        def _attention_pool(self, chunk_tokens):
            """
            Attention-based pooling for chunk representation

            Better than mean pooling as it focuses on important tokens
            """
            if chunk_tokens.size(0) == 0:
                return torch.zeros(self.embed_dim, device=chunk_tokens.device)

            # Compute attention weights based on token importance
            chunk_mean = chunk_tokens.mean(dim=0, keepdim=True)
            attention_scores = torch.sum(chunk_tokens * chunk_mean, dim=-1)
            attention_weights = torch.softmax(attention_scores, dim=0)

            # Weighted sum using attention
            return torch.sum(chunk_tokens * attention_weights.unsqueeze(-1), dim=0)

        def forward(self, input_ids, labels=None):
            """
            Four-level hierarchical forward pass:
            1. Embeddings + Chunking
            2. Chunk-level processing
            3. Global context processing
            4. Hierarchical-sequential fusion
            5. Final sequence processing
            """
            B, L = input_ids.shape

            # ============= Level 1: Embeddings =============
            positions = torch.arange(L, device=input_ids.device).unsqueeze(0).expand(B, -1)
            hidden_states = self.embed(input_ids) + self.pos_embed(positions)
            hidden_states = self.dropout(hidden_states)

            # ============= Level 2: Chunk Processing =============
            chunks = self._create_hierarchical_chunks(hidden_states, input_ids)
            encoded_chunks = self.chunk_encoder(chunks)

            # ============= Level 3: Global Context =============
            global_context = self.global_encoder(encoded_chunks)

            # ============= Level 4: Hierarchical-Sequential Bridge =============
            # Project hierarchical features back to sequence space
            projected_chunks = self.chunk_projection(global_context)

            # Expand chunks back to sequence length
            chunk_expanded = torch.repeat_interleave(projected_chunks, self.chunk_size, dim=1)
            if chunk_expanded.size(1) > L:
                chunk_expanded = chunk_expanded[:, :L, :]
            elif chunk_expanded.size(1) < L:
                padding = torch.zeros(B, L - chunk_expanded.size(1), self.embed_dim,
                                    device=hidden_states.device)
                chunk_expanded = torch.cat([chunk_expanded, padding], dim=1)

            # Gated fusion between hierarchical and sequential features
            gate_input = torch.cat([hidden_states, chunk_expanded], dim=-1)
            gate = self.hierarchical_gate(gate_input)
            combined = gate * chunk_expanded + (1 - gate) * hidden_states

            # ============= Level 5: Final Sequence Processing =============
            final_hidden = self.sequence_encoder(combined)

            # Output projection
            logits = self.output(final_hidden)

            # Compute loss if labels provided
            loss = None
            if labels is not None:
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = labels[..., 1:].contiguous()
                loss_fn = nn.CrossEntropyLoss()
                loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            return logits, loss

        def generate(self, input_ids, max_length=None, num_return_sequences=1, temperature=1.0, do_sample=False, **kwargs):
            """Robust generation method matching other models"""
            self.eval()
            device = input_ids.device

            if max_length is None:
                max_length = input_ids.size(1) + 50

            max_length = min(max_length, 512)
            generated = input_ids.clone()

            top_k = kwargs.get('top_k', 50)
            top_p = kwargs.get('top_p', 1.0)
            repetition_penalty = kwargs.get('repetition_penalty', 1.2)

            with torch.no_grad():
                for _ in range(max_length - input_ids.size(1)):
                    logits, _ = self.forward(generated)
                    next_token_logits = logits[:, -1, :]

                    # Apply repetition penalty
                    if repetition_penalty != 1.0 and generated.size(1) > 0:
                        for i in range(generated.size(0)):
                            for prev_token_idx in set(generated[i, max(0, generated.size(1) - 10):].tolist()):
                                if prev_token_idx < next_token_logits.size(-1):
                                    if next_token_logits[i, prev_token_idx] < 0:
                                        next_token_logits[i, prev_token_idx] *= repetition_penalty
                                    else:
                                        next_token_logits[i, prev_token_idx] /= repetition_penalty

                    next_token_logits = next_token_logits / max(temperature, 1e-8)

                    if do_sample:
                        filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)

                        all_filtered_out = (filtered_logits == -float('Inf')).all(dim=-1)
                        if all_filtered_out.any():
                            for i in range(all_filtered_out.size(0)):
                                if all_filtered_out[i]:
                                    filtered_logits[i, 0] = 1.0

                        filtered_logits = torch.nan_to_num(filtered_logits, nan=-1e9, posinf=1e9, neginf=-1e9)
                        probs = F.softmax(filtered_logits, dim=-1)
                        probs = torch.clamp(probs, min=1e-9)
                        probs = probs / probs.sum(dim=-1, keepdim=True)
                        next_token = torch.multinomial(probs, num_samples=1)
                    else:
                        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)

                    generated = torch.cat([generated, next_token], dim=-1)

                    # Stopping conditions
                    if hasattr(self.tokenizer, 'eos_token_id') and self.tokenizer.eos_token_id is not None:
                        if (next_token == self.tokenizer.eos_token_id).any():
                            break

                    if generated.size(1) >= input_ids.size(1) + 3:
                        if (generated[:, -1] == generated[:, -2]).any() and \
                          (generated[:, -1] == generated[:, -3]).any():
                            break

            return generated

    return PureHNetModel(vocab_size, tokenizer)

##HNet-GPT2 Hybrid: the Novel Architecture

**The main contribution**: Combines hierarchical encoding with proven GPT-2 generation:

### Architecture Philosophy
- **Hierarchical Encoder**: Code-aware adaptive chunking for structure understanding
- **GPT-2 Decoder**: Proven autoregressive generation capabilities  
- **Smart Fusion**: Complexity-aware gating and dynamic scaling

### Key Innovations
1. **Adaptive Chunking**: Code structure boundaries (def/class/if/for) vs fixed windows
2. **Pre-trained Leverage**: Uses GPT-2 blocks instead of training from scratch
3. **Gradual Training**: Strategic unfreezing for optimal learning
4. **Complexity-Aware Integration**: Dynamic fusion based on code complexity

### Why It Works
- **Structure + Generation**: Gets hierarchical understanding AND proven generation
- **Best of Both Worlds**: Combines strengths of both parent architectures
- **Smart Training**: Leverages existing knowledge while adding structure awareness


In [7]:
def create_hnet_gpt2_hybrid(vocab_size, tokenizer):
    """HNet encoder with GPT-2 decoder blocks"""

    class HNetGPT2Hybrid(nn.Module):
        def __init__(self, vocab_size, tokenizer, embed_dim=768, num_chunks=24):
            super().__init__()
            self.tokenizer = tokenizer  # Store as instance variable

            # Use GPT-2's embedding dimension for compatibility
            self.embed_dim = embed_dim

            # Embeddings (matching GPT-2's setup)
            self.embed = nn.Embedding(vocab_size, embed_dim)
            self.pos_embed = nn.Embedding(1024, embed_dim)
            self.dropout = nn.Dropout(0.1)

            # ============= HNet Encoder (from original) =============
            self.num_chunks = 32  # Smaller chunks for better granularity
            self.chunk_size = 512 // self.num_chunks
            self.chunk_overlap = int(self.chunk_size * 0.25)  # 25% overlap


            # Hierarchical chunk encoder (from HNet)
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=12,  # Match GPT-2's attention heads
                dim_feedforward=embed_dim * 4,
                dropout=0.1,
                activation='gelu',
                batch_first=True
            )
            self.chunk_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)

            # ============= GPT-2 Decoder Blocks =============
            # Load GPT-2 configuration and extract decoder blocks
            gpt2_config = GPT2Config(
                vocab_size=vocab_size,
                n_embd=embed_dim,
                n_layer=12,  # Use 12 layers like GPT-2 small
                n_head=12,
                activation_function='gelu_new',
                resid_pdrop=0.1,
                embd_pdrop=0.1,
                attn_pdrop=0.1,
            )

            # Create GPT-2 model and extract transformer blocks
            gpt2_model = GPT2Model(gpt2_config)
            self.gpt2_blocks = gpt2_model.h  # Extract the transformer blocks
            self.ln_f = gpt2_model.ln_f  # Final layer norm


            self.hierarchical_projection = nn.Sequential(
                nn.Linear(embed_dim, embed_dim // 2),
                nn.Dropout(0.1),
                nn.Linear(embed_dim // 2, embed_dim)
            )

            # Add dropout to gating:
            self.hierarchical_gate = nn.Sequential(
                nn.Linear(embed_dim * 2, embed_dim),
                nn.Dropout(0.1),  # Add dropout here
                nn.Sigmoid()
            )

            # Initialize bias to favor original representations
            nn.init.constant_(self.hierarchical_gate[0].bias, -2.0)


            # Output projection
            self.output = nn.Linear(embed_dim, vocab_size)

            # Initialize weights
            self.apply(self._init_weights)

        def _init_weights(self, module):
            if isinstance(module, (nn.Linear, nn.Embedding)):
                module.weight.data.normal_(mean=0.0, std=0.02)
                if isinstance(module, nn.Linear) and module.bias is not None:
                    module.bias.data.zero_()


        def _create_adaptive_chunks(self, hidden_states, input_ids):
            """Create chunks based on code structure boundaries"""
            B, L, D = hidden_states.shape
            chunks = []

            for batch_idx in range(B):
                # Decode tokens back to text to find structure
                try:
                    # Get non-padded tokens
                    tokens = input_ids[batch_idx]
                    text = self.tokenizer.decode(tokens, skip_special_tokens=True)

                    # Find function/class boundaries using simple heuristics
                    boundaries = [0]
                    lines = text.split('\n')
                    current_pos = 0

                    for line in lines:
                        if line.strip().startswith(('def ', 'class ', 'if ', 'for ', 'while ')):
                            # Convert line position back to token position (approximation)
                            char_pos = text.find(line)
                            if char_pos > 0:
                                # Rough token position estimate
                                token_pos = min(int(char_pos * 0.3), L-1)  # Rough char-to-token ratio
                                if token_pos > boundaries[-1] + 10:  # Minimum chunk size
                                    boundaries.append(token_pos)

                    boundaries.append(L)

                except:
                    # Fallback to fixed chunking if parsing fails
                    boundaries = [i * (L // self.num_chunks) for i in range(self.num_chunks + 1)]
                    boundaries[-1] = L

                # Create chunks from boundaries
                batch_chunks = []
                for i in range(len(boundaries) - 1):
                    start, end = boundaries[i], boundaries[i + 1]

                    # Add overlap for non-first chunks
                    if i > 0:
                        start = max(0, start - self.chunk_overlap)

                    # Add overlap for non-last chunks
                    if i < len(boundaries) - 2:
                        end = min(L, end + self.chunk_overlap)

                    if end > start:
                        chunk_tokens = hidden_states[batch_idx, start:end]
                        chunk_repr = torch.mean(chunk_tokens, dim=0)
                        batch_chunks.append(chunk_repr)

                # Pad to consistent number of chunks
                while len(batch_chunks) < self.num_chunks:
                    batch_chunks.append(torch.zeros(D, device=hidden_states.device))

                # Take first num_chunks if we have too many
                batch_chunks = batch_chunks[:self.num_chunks]
                chunks.append(torch.stack(batch_chunks))
            #
            return chunks  # chunks is already a list of tensors for each batch



###


        def forward(self, input_ids, labels=None):
            B, L = input_ids.shape

            # ============= Embeddings =============
            positions = torch.arange(L, device=input_ids.device).unsqueeze(0).expand(B, -1)
            hidden_states = self.embed(input_ids) + self.pos_embed(positions)
            hidden_states = self.dropout(hidden_states)

            # ============= HNet Hierarchical Encoding =============


            # Create adaptive chunks based on code structure
            chunks = self._create_adaptive_chunks(hidden_states, input_ids)
            chunk_tensor = torch.stack(chunks, dim=0)  # dim=0 for batch dimension
            encoded_chunks = self.chunk_encoder(chunk_tensor)


            # ============= GPT-2 Decoder with Cross-Attention =============
            # Before
            # First, apply cross-attention to incorporate hierarchical info
            # Now
            # Expand chunk representations back to sequence length
            chunk_expanded = torch.repeat_interleave(encoded_chunks, self.chunk_size, dim=1)
            if chunk_expanded.size(1) > L:
                chunk_expanded = chunk_expanded[:, :L, :]
            elif chunk_expanded.size(1) < L:
                padding = torch.zeros(B, L - chunk_expanded.size(1), self.embed_dim, device=hidden_states.device)
                chunk_expanded = torch.cat([chunk_expanded, padding], dim=1)

            # Project hierarchical features
            hierarchical_features = self.hierarchical_projection(chunk_expanded)

            # Learned gating
            gate_input = torch.cat([hidden_states, hierarchical_features], dim=-1)
            gate = self.hierarchical_gate(gate_input)

            # With this (residual scaling):
            gated_hierarchical = gate * hierarchical_features + (1 - gate) * hidden_states

            complexity_score = torch.mean(gate, dim=-1, keepdim=True)  # [B, L, 1]
            alpha = 0.8 + 0.2 * complexity_score  # Dynamic 0.8-1.0 range
            hidden_states = alpha * hidden_states + (1 - alpha) * gated_hierarchical


            # Apply GPT-2 transformer blocks
            for block in self.gpt2_blocks:
                outputs = block(hidden_states)
                hidden_states = outputs[0]

            # Final layer norm
            hidden_states = self.ln_f(hidden_states)

            # Output projection
            logits = self.output(hidden_states)

            # Compute loss
            loss = None
            if labels is not None:
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = labels[..., 1:].contiguous()
                loss_fn = nn.CrossEntropyLoss()
                loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            return logits, loss

            ##
        def generate(self, input_ids, max_length=None, num_return_sequences=1, temperature=1.0, do_sample=False, **kwargs):
            """FIXED: Properly reference self.tokenizer"""
            self.eval()
            device = input_ids.device

            if max_length is None:
                max_length = input_ids.size(1) + 50

            max_length = min(max_length, 512)
            generated = input_ids.clone()

            top_k = kwargs.get('top_k', 50)
            top_p = kwargs.get('top_p', 1.0)
            repetition_penalty = kwargs.get('repetition_penalty', 1.2)

            with torch.no_grad():
                for _ in range(max_length - input_ids.size(1)):
                    logits, _ = self.forward(generated)
                    next_token_logits = logits[:, -1, :]

                    # Apply repetition penalty
                    if repetition_penalty != 1.0 and generated.size(1) > 0:
                        for i in range(generated.size(0)):
                            for prev_token_idx in set(generated[i, max(0, generated.size(1) - 10):].tolist()):
                                if prev_token_idx < next_token_logits.size(-1):
                                    if next_token_logits[i, prev_token_idx] < 0:
                                        next_token_logits[i, prev_token_idx] *= repetition_penalty
                                    else:
                                        next_token_logits[i, prev_token_idx] /= repetition_penalty

                    next_token_logits = next_token_logits / max(temperature, 1e-8)

                    if do_sample:
                        filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)

                        all_filtered_out = (filtered_logits == -float('Inf')).all(dim=-1)
                        if all_filtered_out.any():
                            for i in range(all_filtered_out.size(0)):
                                if all_filtered_out[i]:
                                    filtered_logits[i, 0] = 1.0

                        filtered_logits = torch.nan_to_num(filtered_logits, nan=-1e9, posinf=1e9, neginf=-1e9)
                        probs = F.softmax(filtered_logits, dim=-1)
                        probs = torch.clamp(probs, min=1e-9)
                        probs = probs / probs.sum(dim=-1, keepdim=True)
                        next_token = torch.multinomial(probs, num_samples=1)
                    else:
                        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)

                    generated = torch.cat([generated, next_token], dim=-1)

                    # FIXED: Use self.tokenizer instead of tokenizer
                    if self.tokenizer.eos_token_id is not None and (next_token == self.tokenizer.eos_token_id).any():
                        break

                    if generated.size(1) >= input_ids.size(1) + 3:
                        if (generated[:, -1] == generated[:, -2]).any() and \
                          (generated[:, -1] == generated[:, -3]).any():
                            break

            return generated

            ##




    return HNetGPT2Hybrid(vocab_size, tokenizer)

 # Section 5: Training Infrastructure


Advanced training setup with:
- **Gradual Unfreezing**: Strategic component unfreezing for hybrid model
- **Warmup Scheduling**: Learning rate warmup + cosine decay
- **Gradient Clipping**: Stable training for large models
- **Model Checkpointing**: Automatic saving to Google Drive

In [8]:
def train_model(model, train_loader, device, epochs=25, lr=1e-4, model_name="Model"):
    """Train model with warmup + freezing strategy"""

    # Phase 1: Freeze GPT-2 blocks
    if hasattr(model, 'gpt2_blocks'):
        for block in model.gpt2_blocks:
            for param in block.parameters():
                param.requires_grad = False
        print("Phase 1: GPT-2 blocks frozen, training hierarchical components only")

    # Calculate warmup steps
    warmup_epochs = 2
    total_steps = len(train_loader) * epochs
    warmup_steps = len(train_loader) * warmup_epochs

    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr * 0.01,  # Start with very small LR
        betas=(0.9, 0.95), weight_decay=0.05
    )

    # Warmup + Cosine scheduler
    from transformers import get_linear_schedule_with_warmup
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    model.train()
    losses = []
    step = 0

    for epoch in range(epochs):
        # Unfreezing schedule (same as before)
        if hasattr(model, 'gpt2_blocks') and epoch == 8:
            print("Phase 2: Unfreezing last 6 GPT-2 blocks")
            for block in model.gpt2_blocks[-6:]:
                for param in block.parameters():
                    param.requires_grad = True

        elif hasattr(model, 'gpt2_blocks') and epoch == 16:
            print("Phase 3: Unfreezing all GPT-2 blocks")
            for block in model.gpt2_blocks:
                for param in block.parameters():
                    param.requires_grad = True
            # Update optimizer
            optimizer = torch.optim.AdamW(
                filter(lambda p: p.requires_grad, model.parameters()),
                lr=lr * 0.3,
                betas=(0.9, 0.95), weight_decay=0.05
            )

        epoch_loss = 0
        epoch_batches = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for input_ids, labels in pbar:
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            try:
                logits, loss = model(input_ids, labels=labels)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

                # Step scheduler every batch during warmup
                if step < total_steps:
                    scheduler.step()
                    step += 1

                epoch_loss += loss.item()
                epoch_batches += 1

                # Show current LR in progress bar
                current_lr = optimizer.param_groups[0]['lr']
                pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'lr': f'{current_lr:.6f}'
                })

            except Exception as e:
                print(f"Training error: {e}")
                continue

        avg_loss = epoch_loss / max(epoch_batches, 1)
        losses.append(avg_loss)
        print(f"   Epoch {epoch+1}: Loss = {avg_loss:.4f}, LR = {current_lr:.6f}")

    # Save model
    save_path = f"/content/drive/MyDrive/hnet_gpt2_models/{model_name.replace(' ', '_')}.pt"
    torch.save(model.state_dict(), save_path)
    print(f"Saved model to {save_path}")

    return {'losses': losses, 'final_loss': losses[-1] if losses else float('inf')}




def evaluate_model(model, test_loader, device):
    """Evaluate model on test data"""

    model.eval()
    total_loss = 0
    total_tokens = 0

    with torch.no_grad():
        for input_ids, labels in tqdm(test_loader, desc="Evaluating"):
            input_ids = input_ids.to(device)
            labels = labels.to(device)

            try:
                logits, loss = model(input_ids, labels=labels)

                # Count tokens
                mask = (labels != -100).float()
                total_loss += loss.item() * mask.sum().item()
                total_tokens += mask.sum().item()

            except Exception as e:
                print(f"Evaluation error: {e}")
                continue

    if total_tokens == 0:
        return float('inf')

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(min(avg_loss, 100))  # Cap to avoid overflow

    return perplexity




def print_results(results):
    """Print benchmark results"""

    print(f"\nHYBRID ARCHITECTURE RESULTS")
    print("=" * 60)

    # Sort by perplexity
    sorted_results = sorted([r for r in results if r['test_perplexity'] != float('inf')],
                           key=lambda x: x['test_perplexity'])

    print(f"FINAL RANKING:")
    print(f"{'Rank':<5} {'Model':<30} {'Perplexity':<12} {'Params':<10} {'Time(s)':<10}")
    print("-" * 70)

    for i, result in enumerate(sorted_results, 1):
        print(f"{i:<5} {result['name']:<30} {result['test_perplexity']:<12.2f} "
              f"{result['parameters']:>9,} {result['training_time']:<10.1f}")

    # Compare hybrid vs pure GPT-2
    hybrid_result = next((r for r in results if 'Hybrid' in r['name']), None)
    gpt2_result = next((r for r in results if 'Pure' in r['name']), None)

    if hybrid_result and gpt2_result:
        improvement = ((gpt2_result['test_perplexity'] - hybrid_result['test_perplexity']) /
                      gpt2_result['test_perplexity']) * 100

        print(f"\nHYBRID ARCHITECTURE ANALYSIS:")
        print(f"HNet-GPT2-Hybrid:  {hybrid_result['test_perplexity']:.2f} perplexity")
        print(f"Pure GPT-2:        {gpt2_result['test_perplexity']:.2f} perplexity")
        print(f"Improvement:       {improvement:.1f}%")

        print(f"\nARCHITECTURAL INSIGHTS:")
        print(f"• HNet hierarchical encoding: Provides chunk-level context")
        print(f"• GPT-2 decoder blocks: Leverages pretrained architecture")
        print(f"• Cross-attention fusion: Combines hierarchical + sequential")
        print(f"• Real MBPP data: Tests on actual code understanding tasks")

# Section 6: Enhanced Code-Focused Evaluation

**The main experiment**: Comprehensive evaluation of all three approaches:

## Research Question
Which approach works best for code generation?
1. **Pure hierarchical processing** (Pure HNet)
2. **Pure sequential processing** (Pure GPT-2)  
3. **Hybrid hierarchical-sequential** (Our HNet-GPT2)


## Evaluation Metrics
* **Syntax Validity**: Whether generated code parses correctly
* **Pattern Recognition**: Ability to complete common code patterns (e.g., `a + b`, `n % 2 == 0`)
* **Code Token Usage**: Percentage of actual code tokens vs. natural language
* **Composite Score**: Weighted combination of all metrics

## Test Cases
* Basic arithmetic operations (`def add(a, b): return ___`)
* Boolean logic (`def is_even(n): return ___`)
* Conditional statements (`def max_two(x, y): if x > y: return ___`)
* Simple expressions (`def square(n): return ___`)
* Comparison operations (`def is_positive(x): return ___`)


## Experimental Setup
- **Training**: 25 epochs with adaptive learning rate
- **Generation Strategies**: Greedy, Conservative (temp=0.3), Creative (temp=0.7)
- **Hardware**: GPU acceleration with CUDA
- **Focus**: Practical code completion rather than abstract metrics


In [9]:
# Run the enhanced evaluation on pre-trained models

enhanced_results = show_enhanced_results()

ENHANCED EVALUATION OF EXISTING MODELS
Testing your trained models with code-focused evaluation


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Identifying code-relevant tokens...
Identified 141 code-relevant tokens
   Test coverage 'a + b': 3/3 tokens (100.0%)

TESTING: Pure HNet
Building Pure HNet Architecture:
Vocabulary: 50,257 tokens
Embedding dimension: 768
Chunking strategy: 32 chunks of size 16
Chunk overlap: 4 tokens
Total parameters: 193,798,993
Loaded existing model weights

CODE-FOCUSED EVALUATION: Pure HNet

Test 1: Basic addition
   Prompt: def add(a, b):
    return
   Generated: ''
No generation produced

Test 2: Boolean logic
   Prompt: def is_even(n):
    return
   Generated: ''
No generation produced

Test 3: Conditional logic
   Prompt: def max_two(x, y):
    if x > y:
        return
   Generated: ''
No generation produced

Test 4: Simple expression
   Prompt: def square(n):
    return
   Generated: ''
No generation produced

Test 5: Comparison
   Prompt: def is_positive(x):
    return
   Generated: ''
No generation produced

CODE-FOCUSED RESULTS FOR Pure HNet:
Syntax Validity: 0/5 (0.0%)
Pattern Recognition